In [1]:
# Goal : Get tittle of every book with a 2 star rating

In [2]:
import requests
import bs4

In [3]:
# 1~1000까지가 한 페이지에 있지 않다
# 다음 페이지로 넘어가는 과정을 loop로 구현해야 한다
# 먼저 첫 페이지의 URL이다
'https://books.toscrape.com/'
# 두 번째 페이지의 URL
'https://books.toscrape.com/catalogue/page-2.html'

'https://books.toscrape.com/catalogue/page-2.html'

In [4]:
# 첫 페이지를 이렇게 바꿀 수 있겠다
'https://books.toscrape.com/catalogue/page-1.html'

'https://books.toscrape.com/catalogue/page-1.html'

In [5]:
base_url = 'https://books.toscrape.com/catalogue/page-{}.html'

In [6]:
base_url.format('20')

'https://books.toscrape.com/catalogue/page-20.html'

In [7]:
page_number = 20
base_url.format(page_number)

'https://books.toscrape.com/catalogue/page-20.html'

사이트에서 별점에 마우스를 올리고 검사를 해보면 class icon-star이 있다
우리는 여기서 별점이 2개짜리인 녀석들의 책 제목을 모으려 한다. 그런데 이 제목과
별점들은 모두 class product_pod 안에 묶여있다. 그러므로 우리는 product_pod 전체를
가져온 다음 별점을 클래스로 걸러내야 한다

In [8]:
res = requests.get(base_url.format(1))

In [9]:
soup = bs4.BeautifulSoup(res.text, 'lxml')

In [11]:
len(soup.select(".product_pod")) # 한 페이지에 책이 20권 있다는 소리

20

In [12]:
products = soup.select(".product_pod")

In [13]:
example = products[0]

In [14]:
example

<article class="product_pod">
<div class="image_container">
<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [15]:
# example에 벼럼이 두 개인지 확인하는 방법
# 1. 문자열 변환 
# 문자열로 변환한 후 'star-rating Two'가 안에 있는지 확인한다
str(example)

'<article class="product_pod">\n<div class="image_container">\n<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>\n</div>\n<p class="star-rating Three">\n<i class="icon-star"></i>\n<i class="icon-star"></i>\n<i class="icon-star"></i>\n<i class="icon-star"></i>\n<i class="icon-star"></i>\n</p>\n<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>\n<div class="product_price">\n<p class="price_color">Â£51.77</p>\n<p class="instock availability">\n<i class="icon-ok"></i>\n    \n        In stock\n    \n</p>\n<form>\n<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>\n</form>\n</div>\n</article>'

In [16]:
'star-rating Two' in str(example)

False

In [17]:
# 2. beautifulsoup를 통해 찾기 , 주의점: 찾으려는 것에 공백이 있으면 .으로 채운다
example.select(".star-rating.Three")

[<p class="star-rating Three">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>]

In [19]:
[] == example.select(".star-rating.Two")

True

In [20]:
# a안에 들어있는 내용 중 첫 번째는 책의 이미지이고 두 번재는 책의 제목이다
example.select('a')

[<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>,
 <a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>]

In [21]:
example.select('title')

[]

In [23]:
example.select('a')[1]

<a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [24]:
type(example.select('a')[1])

bs4.element.Tag

In [25]:
example.select('a')[1]['title'] # 딕셔너리를 통해 해결가능

'A Light in the Attic'

In [26]:
 two_star_titles = []
    
for n in range(1,51):
    
    scrape_url = base_url.format(n)
    res = requests.get(scrape_url)
    
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    books = soup.select(".product_pod")
    
    for book in books:
        #if 'star-rating Two' in str(book)# 두 번째 방법    
        if len(books.select('.star-rating.Two')) != 0:
            book_title = book.select('a')[1]['title']
            two_star_titles.append(book_title)

AttributeError: ResultSet object has no attribute 'select'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [27]:
 two_star_titles = []
    
for n in range(1,51):
    
    scrape_url = base_url.format(n)
    res = requests.get(scrape_url)
    
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    books = soup.select(".product_pod")
    
    for book in books:
        #if 'star-rating Two' in str(book)# 두 번째 방법    
        if len(books.select('.star-rating.Two')) != 0:
            book_title = book.select('a')[1]['title']
            two_star_titles.append(book_title)

AttributeError: ResultSet object has no attribute 'select'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?